In [1]:
import gensim
from gensim.models.keyedvectors import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
import wikipedia as wiki
import re

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
glove2word2vec(glove_input_file = "D:/glove.6B/glove.6B.300d.txt", word2vec_output_file = "D:/gensim_glove_vectors.txt")
glove_model = KeyedVectors.load_word2vec_format("D:/gensim_glove_vectors.txt", binary=False)
glove_model.init_sims(replace=True)

In [4]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet 
from itertools import product

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
def wn_similarity(x, y):
    for sense1, sense2 in product(x, y):
        d = wordnet.wup_similarity(sense1, sense2)
        if d != None:
            wn_sim = d
        else:
            wn_sim = 0
    return wn_sim

In [38]:
#wv-wk
dataset = open("C:/Users/1/Desktop/validation.txt", encoding = 'utf-8').readlines()
K, m = [], []
for d in range(len(dataset)):
    try:
        lin = dataset[d].strip().split(',')
            
        target_sim = glove_model.similarity(lin[0], lin[2])
        comp_sim = glove_model.similarity(lin[1], lin[2])
        targ_comp_sim = glove_model.similarity(lin[0], lin[1])
            
        try:
            sum_lin_0 = wiki.summary(lin[0])
        except wiki.exceptions.DisambiguationError as e:
            sum_lin_0 = wiki.summary(e.options[0])
        try:
            sum_lin_1 = wiki.summary(lin[1])
        except wiki.exceptions.DisambiguationError as e:
            sum_lin_1 = wiki.summary(e.options[0])
        lin_0_wiki = len(re.findall(lin[2], sum_lin_0))
        lin_1_wiki = len(re.findall(lin[2], sum_lin_1))
                
        K.append((target_sim, comp_sim, target_sim-comp_sim, targ_comp_sim, lin_0_wiki, lin_1_wiki, lin_0_wiki-lin_1_wiki))
        m.append(lin[3])
    except:
        continue

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [39]:
print (len(K))

2696


In [40]:
K_train = K[0:1696]
m_train = m[0:1696]
K_test = K[1696:]
m_test = m[1696:]

In [34]:
#+wordnet
dataset = open("C:/Users/1/Desktop/validation.txt", encoding = 'utf-8').readlines()
X, y = [], []
for d in range(len(dataset)):
    try:
        lin = dataset[d].strip().split(',')
            
        target_sim = glove_model.similarity(lin[0], lin[2])
        comp_sim = glove_model.similarity(lin[1], lin[2])
        targ_comp_sim = glove_model.similarity(lin[0], lin[1])
            
        try:
            sum_lin_0 = wiki.summary(lin[0])
        except wiki.exceptions.DisambiguationError as e:
            sum_lin_0 = wiki.summary(e.options[0])
        try:
            sum_lin_1 = wiki.summary(lin[1])
        except wiki.exceptions.DisambiguationError as e:
            sum_lin_1 = wiki.summary(e.options[0])
        lin_0_wiki = len(re.findall(lin[2], sum_lin_0))
        lin_1_wiki = len(re.findall(lin[2], sum_lin_1))
        
        syns0 = wordnet.synsets(lin[0])
        syns1 = wordnet.synsets(lin[1])
        syns2 = wordnet.synsets(lin[2])
        first_wn = wn_similarity(syns0, syns2)
        second_wn = wn_similarity(syns1, syns2)
        between_themselves_wn = wn_similarity(syns0, syns1)
                
        X.append((target_sim, comp_sim, target_sim-comp_sim, targ_comp_sim, lin_0_wiki, lin_1_wiki, lin_0_wiki-lin_1_wiki, first_wn, second_wn, between_themselves_wn))
        y.append(lin[3])
    except:
        continue

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [35]:
print (len(X))

2691


In [36]:
X_train = X[0:1691]
y_train = y[0:1691]
X_test = X[1691:]
y_test = y[1691:]

In [41]:
#wv-wk
neigh = KNeighborsClassifier(n_neighbors=20, p=1)
neigh.fit(K_train, m_train)

predictions = neigh.predict(K_test)
print(classification_report(m_test, predictions))

             precision    recall  f1-score   support

          0       0.70      0.69      0.70       508
          1       0.69      0.70      0.69       492

avg / total       0.70      0.70      0.70      1000



In [42]:
#+wordnet
neigh = KNeighborsClassifier(n_neighbors=20, p=3)
neigh.fit(X_train, y_train)

predictions = neigh.predict(X_test)
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.65      0.65      0.65       508
          1       0.64      0.64      0.64       492

avg / total       0.65      0.65      0.65      1000



In [43]:
#wv_wk
ran_for = RandomForestClassifier(n_estimators=15, 
                                 max_features=None, random_state=42, max_depth=5)
ran_for.fit(K_train, m_train)

predictions = ran_for.predict(K_test)
print(classification_report(m_test, predictions))

             precision    recall  f1-score   support

          0       0.73      0.65      0.69       508
          1       0.67      0.75      0.71       492

avg / total       0.70      0.70      0.70      1000



In [44]:
#+wordnet
ran_for = RandomForestClassifier(n_estimators=15, 
                                 max_features=None, random_state=42, max_depth=5)
ran_for.fit(X_train, y_train)

predictions = ran_for.predict(X_test)
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.73      0.64      0.69       508
          1       0.67      0.76      0.71       492

avg / total       0.70      0.70      0.70      1000

